In [93]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
load_dotenv()

True

## load variables - bunds
- data folder
- cleaned_df - for only bunds
- json template
- 

In [94]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)

#lambda function to filter for bunds
#for each row, is it a list/str, then is 'bunds' in it, if so, return the df.
cleaned_df = dff[
    dff["land_use_selection-intervention"]
    .apply(lambda x: isinstance(x, (list, str)) and "soil_bunds" in x)
]

#bunds datasets
bunds_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'outputs','cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")
with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## check bunds count

In [95]:
x = dff["land_use_selection-intervention"]
count = 0
for idx, i in enumerate(list(x)):
    if 'soil_bunds' in i:
        count += 1
        #ic(i, count)

ic('soil_bunds count',len(cleaned_df))

ic| 'soil_bunds count', len(cleaned_df): 12


('soil_bunds count', 12)

## enumerators

In [96]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

#ic(enums)

In [97]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    #ic(farming_entity_list)
    

# plot-soil_bunds details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
    - every matching rows values are appended to a list(of all crops for that plot)
- clean 'and' from the crop types



In [98]:
import uuid
import copy
import re

plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #get only child row for this key
    matching_child_rows = bunds_df[bunds_df['PARENT_KEY'] == i['KEY']]

    #holds every row's crops
    all_crops = []
    """----------------------------------CROP STATUS ----------------------------------"""
    #not in child table, so skipped
        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    
    def parse_geoshape(geoshape_str):
        #parse a geoshape string into a list of point dictionaries
        points = []
        parts = geoshape_str.split(';')

        for part in parts:
            part = part.strip()
            if not part:
                continue

            lat, lon, alt, acc = map(float, part.split(' '))

            points.append({
                "longitude": lon,
                "latitude": lat,
                "altitude": alt,
                "accuracy": acc
            })

        return points

    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


## project

In [99]:
projects_list = []

names =[]
for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']
    
    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93
    
    names.extend([name])
    
    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })

ic(set(names))


ic| set(names): {'TERRA', 'BRCiS III'}


{'BRCiS III', 'TERRA'}

## Bunds
- has only two child rows
- with only 1 grass info


In [100]:
bunds_list = []

#dictionary to map odk values to RA app schema values
mapping_dict = {
    "STICKS_BRANCH":"STICKS_PROTECTION",
    "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
    "INVASIVE_SPECIES":"INVASIVE_CONTROL",
    "WEEDING":"WEEDING",
    "WATERING":"WATERING",
    "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
}

for idx, c in enumerate(records):
    cureent_status_list = []
    all_grasses = []

    """-----------------------------------BUNDS CHILD DETAILS-------------------------------------"""

    total_num_grasses_planted = 0
    total_kg_grass_seeds = 0.0
    grass_present = []
    mngmts = []
    matching_child_rows = bunds_df[bunds_df['PARENT_KEY'] == c['KEY']]

    idxx =0
    for x in matching_child_rows.to_dict(orient='records'):
        total_num_grasses_planted += 0 if pd.isna(x["grass_bunds-num_grass_species"]) else x["grass_bunds-num_grass_species"]
        total_kg_grass_seeds += 0 if pd.isna(x["grass_bunds-kg_grass_seeds"]) else x["grass_bunds-kg_grass_seeds"]
        grass_present.append(True if str(x["tree_bunds-planted_choice3"]).strip() in ["both", "grasses"] else False)


        grass_status_list = []
        """---------------GRASS_ESTABLISHMENT------------"""
        #get all grasses 
        if x["tree_bunds-planted_choice3"] in ["both", "grasses"]:
            grass_species_list = str(x["grass_bunds-bunds_grass_species"]).split()
            #default
            scientific_name = ""
            local_name = ""

            for idx2 ,raw in enumerate(grass_species_list):
                #case 1: others: not there

                #case2: ends with other and also has nan in other colun

                #split scientific and local names
                if "(" in raw and ")" in raw:
                    #scientific name cleaning
                    parts = raw.split("(")[0].strip().replace("_", " ").split()
                    if len(parts) == 2:
                        scientific_name = f"{parts[0].capitalize()} {parts[1].lower()}"
                    else:
                        scientific_name = raw.split("(")[0].strip().replace("_", " ")

                    #local name cleaning
                    paarts = raw.split("(")[1].split(")")[0].strip().replace("_", " ").split()
                    if len(paarts) == 2:
                        local_name = f"{paarts[0].capitalize()} {paarts[1].lower()}"
                    else:
                        local_name = raw.split("(")[1].split(")")[0].strip().replace("_", " ").replace(",", " ")#[1].lower()
                else:
                    scientific_name = raw.strip()

                if local_name != 'nan':
                    grass_current_status = {
                        "id": idx2 + 1,
                        "local_name": local_name,
                        "scientific_name": scientific_name,
                        "econtrol_status_id": 1
                    }
            
                #append the current status for ths record to the empty list
                grass_status_list.append(grass_current_status)
            ic(grass_status_list)
        

        """--------------------CURRENT STATUS--------------------"""
        idxx += 1
        currentStatus= {
            "id": idxx,
            "key":c["KEY"],            
            "herbaceous_cover": "",
            "visible_erosion": True,
            "erosion_type": [
                "OTHER"
            ],
            "other_erosion_type": "NONE",
            "is_intervention_effective": True,
            "longitude": x["gps-Longitude"],
            "latitude": x["gps-Latitude"],
            "altitude": x["gps-Altitude"],
            "accuracy": x["gps-Accuracy"],
            "photo_url": x["photo"],
            "comments": f"agric_field_state {x["managements_bunds-agricultural_field_state"] if pd.notna(x["managements_bunds-agricultural_field_state"]) else "MIGRATED_NOT_KNOWN"} state {c["geography-state_name"].replace("â€¦", "").replace(".", "").replace("..", "")} community {c['geography-community_name']}",
            "econtrolId": 1,
            "grassCurrentStatus":grass_status_list,
            "treeCurrentStatus":[]
        }
        
        #add every status to the to serve as establishment        
        all_grasses.extend(grass_status_list)

        cureent_status_list.append(currentStatus)


    """-----------------------------------BUNDS DETAILS-------------------------------------"""

    bunds_details = {
        "id": idx + 1,
        "key":c["KEY"],
        "erosion_control_type": "Bunds",
        "established_date": datetime.strptime(c['bunds-bunds_details-date'], "%d/%m/%Y").strftime("%Y-%m"),
        "material_used": [
            "Soil"
        ],
        "other_material_used": "",
        "who_established": ["MALE", "FEMALE"] if c["bunds-bunds_details-manages"] == 'both' else "MALE" if c["bunds-bunds_details-manages"]=="male" else "",
        "other_who_established": "YOUTH" if c["bunds-bunds_details-youth_manages"] == 'yes' else "",
        "total_interventions": c["bunds-bunds_details-total"],
        "length": c["bunds-bunds_details-length"],
        "width": c["bunds-bunds_details-width"],
        "depth": c["bunds-bunds_details-depth"],
        "vertical_spacing": 0,
        "horizontal_spacing": 0,
        "labor_payment": c["bunds-bunds_details-paid_unpaid_labor"],        
        "has_grass": any(grass_present),
        "total_number_grasses_planted": total_num_grasses_planted,
        "kg_grass_seeds_planted": total_kg_grass_seeds,
        "has_trees": False,
        "total_number_different_trees_planted": 0,
        "total_number_trees_planted": 0,
        "total_number_trees_survived": 0,
        "management_practices":  [],
        "other_managements": " ",
        "usages":["MIGRATED_NOT_KNOWN"],
        "other_usages": "",
        "rangelandEntryId": 2,
        "currentStatus": cureent_status_list,
        "grassEstablishment": all_grasses,
        "treeEstablishment":[]  
    }

    bunds_list.append(bunds_details)
    


bunds_econtrol = [{"econtrol": [i], "key" :i["key"]} for i in bunds_list]
ic(bunds_econtrol)

ic| grass_status_list: [{'econtrol_status_id': 1,
                         'id': 1,
                         'local_name': 'Jarbi',
                         'scientific_name': 'Dactyloctenium'}]
ic| bunds_econtrol: [{'econtrol': [{'currentStatus': [{'accuracy': 4.961,
                                                       'altitude': 359.60205078125,
                                                       'comments': 'agric_field_state '
                                                                   'MIGRATED_NOT_KNOWN state '
                                                                   'southwest community waajid',
                                                       'econtrolId': 1,
                                                       'erosion_type': ['OTHER'],
                                                       'grassCurrentStatus': [{'econtrol_status_id': 1,
                                                                               'id': 1,
                     

[{'econtrol': [{'id': 1,
    'key': 'uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45',
    'erosion_control_type': 'Bunds',
    'established_date': '2025-03',
    'material_used': ['Soil'],
    'other_material_used': '',
    'who_established': 'MALE',
    'other_who_established': '',
    'total_interventions': 0.0,
    'length': 0.0,
    'width': 0.0,
    'depth': 0.0,
    'vertical_spacing': 0,
    'horizontal_spacing': 0,
    'labor_payment': 'paid',
    'has_grass': True,
    'total_number_grasses_planted': 1.0,
    'kg_grass_seeds_planted': 50.0,
    'has_trees': False,
    'total_number_different_trees_planted': 0,
    'total_number_trees_planted': 0,
    'total_number_trees_survived': 0,
    'management_practices': [],
    'other_managements': ' ',
    'usages': ['MIGRATED_NOT_KNOWN'],
    'other_usages': '',
    'rangelandEntryId': 2,
    'currentStatus': [{'id': 1,
      'key': 'uuid:366f2dfd-c43f-4b67-b8a2-b5056ad18b45',
      'herbaceous_cover': '',
      'visible_erosion': True,
 

In [101]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(bunds_econtrol, f, indent=4)

## merge components into one json

In [102]:
# component lists
lists = [farming_entity_list, bunds_econtrol,plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    #match enumerators
    for l in enums:
        if i['key'] == l['key']:
            i['enumerator'] = l


    i["boma"] = []
    i["cgrazing"] = []
    i["microcatchment"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["iremoval"] = []
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']
    #match enumerator id
    for e in enums:
        if i['key'] == e['key']:
            i['enumeratorId'] = e['id']


ic(len(final))


ic| len(lst): 12
ic| len(lst): 12
ic| len(lst): 12
ic| len(lst): 12
ic| len(final): 12


12

In [103]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up keys

In [104]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1



In [105]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)

In [106]:
for i in enumerate(finaal):
    if len(i[1].keys()) != 23:
        ic("ERROR", len(i[1].keys()))
    else:
        ic("✅")

ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
ic| '✅'
